In [1]:
import sys
sys.path.append("../")

In [2]:
import torch
import numpy as np
from utils import get_ground_truth_bnn, evaluate_test_nll, get_param_dim
from tqdm import tqdm

device = torch.device('cpu')
torch.set_default_tensor_type('torch.FloatTensor')

/Users/wenlinchen/miniconda3/envs/rdgs/lib/python3.9/site-packages/torch/__init__.py:614: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/tensor/python_tensor.cpp:453.)
  _C._set_default_tensor_type(t)


In [3]:
d_x=20
d_h=25
s_n=0.1
dim = get_param_dim(d_x, d_h)
x_train, x_test, y_train, y_test, param_true, energy = get_ground_truth_bnn(d_x, d_h, s_n)

print("true_nll:", evaluate_test_nll(param_true, x_test, y_test, d_x, d_h, s_n))

true_nll: -0.8763405084609985


In [4]:
# smapler seed
seed = 1

np.random.seed(seed)
torch.manual_seed(seed)


In [5]:
param = torch.randn([1, dim]).to(device)
param.requires_grad = True

for i in tqdm(range(750000)):
    loss = energy(param) / x_train.shape[0]
    loss.backward()
    param.data = param.data - 0.03 * param.grad.data
    param.grad.data = torch.zeros_like(param.grad.data)

nll = evaluate_test_nll(param, x_test, y_test, d_x, d_h, s_n)
    

100%|██████████| 750000/750000 [02:47<00:00, 4485.88it/s]


In [6]:
print("map nll:", nll)

map nll: 0.7427642345428467
